In [22]:
using POMDPs # for MDP type
using DiscreteValueIteration
using POMDPPolicies
using POMDPModelTools #for sparse cat 
using Parameters
using Random
using Plots; default(fontfamily="Computer Modern", framestyle=:box) # LaTex-style
using QuickPOMDPs
using Distributions 

In [23]:
Random.seed!(123)

MersenneTwister(123)

**Environment Parameters**

In [24]:
# moving below becasue need state defined...

**States**

In [25]:
# A state s in the evacuation problem is a discrete set of 4 values: 
# chairs remaining, time remaining, family size, and visa status
struct State
    c::Int # chairs remaining 
    t::Int # time remaining 
    f::Int # family size 
    v::Int # visa status 
end 

In [26]:
# assume families normally distributed as integers around average family size in afghanistan of 8. 
# Truncated to include 1 or greater and rounded to be integers 
@with_kw struct EvacuationParameters
    # need the probability that a particular family shows up at the door....
    family_sizes::Vector{Int} =  [trunc(Int, x) for x in rand(TruncatedNormal(3, 1, 1, Inf), 10)] 
    visa_status::Vector{Int} = [-4, -3, -2, 0, 1, 2, 3, 4] #TODO: map to various status 
    capacity::Int = 20
    time::Int = 60
    size::Tuple{Int, Int} = (length(visa_status), length(family_sizes)) # size of grid 
    p_transition::Real = 0.8 # don't we always transition into this since time moves forward? I'm confused... 
    null_state::State = State(0, 0, 0 ,0) # is there someway to do this???
end


EvacuationParameters

In [27]:
params = EvacuationParameters(); 

LoadError: cannot assign a value to variable StatsBase.params from module Main

In [28]:
# The state space S for the evacuation problem is the set of 
# all combinations 
# Question: Do I need the end state here? 
#𝒮 = [[State(c, t, f, v) for  c=1:params.capacity, t=1:params.time, f=1:params.size[2], v=1:params.size[1]]]
𝒮 = []

for c in 0:params.capacity # capacity ends at 0 
    for t in 0:params.time # time ends at 0 
        for f in params.family_sizes # family size here we should have the ACTUAL family sizes 
            for v in params.visa_status # actual visa statuses 
                new = State(c, t, f, v) 
                𝒮 = [𝒮; new]
            end
        end        
    end
end
𝒮 = [𝒮; params.null_state]


LoadError: type #params has no field capacity

In [29]:
@show 𝒮

𝒮 = Any[]


Any[]

In [30]:
@show size(𝒮)
#@show size[1,1,1,1]
#(20, 60, 10, 10)
#length(𝒮[1]) = 120000 = 100*60*10*10

size(𝒮) = (0,)


(0,)

**Actions**

In [31]:
# the possible actions are whether accept or reject a family at the gate 
@enum Action REJECT ACCEPT

In [32]:
𝒜 = [REJECT, ACCEPT]

2-element Vector{Action}:
 REJECT::Action = 0
 ACCEPT::Action = 1

In [33]:
length(𝒜)

2

In [34]:
v′ = rand(params.visa_status, 1)

LoadError: type #params has no field visa_status

In [35]:
v′ = rand(params.visa_status, 1)[1]

LoadError: type #params has no field visa_status

In [36]:
# only inbounds if room for the family [assuming would not separate even though might]
# and if time is available to enter the airport 
validtime(s::State) = 0 ≤ s.t 

validtime (generic function with 1 method)

In [37]:
validcapacity(s::State) = 0 ≤ s.c 

validcapacity (generic function with 1 method)

**Transition Function** 

In [38]:
#XXXXXXXXXXXXXXXXXXXXXXXXXNote from Lilian:XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# check out https://juliapomdp.github.io/POMDPModelTools.jl/stable/distributions/
# we are supposed to return a sparsecat structure with all the states and the prob of going to them next
# so i think this is incorrect because we seem to be not returning that
# SparseCat(values, probabilities) to create a sparse categorical distribution.
#textbook page 148

#Other notes:
#T = P(S' | S, A), so the probs are always 100% for each action 

#I figured it out, just havent coded it yet, just have to send back a SparseCat with 
# if action = accept, new state (S') to be time -1 and seats -1 100% and the rest 0%
# if action = reject, send back exact same state space (s') but with time - 1 100% of the time

#this is assuming an accept means 100% you get on the plane and reject means you can't sneak onto it. I 
#I think it might be realistic to make an accept give you a 80% chance of actually getting a seat if accepted.
#i read your article and it seemed like it was so disorganized that some of the accepted people didnt 
#make it. 

#need to add global rep of all the states so we can check it

# think about how to fix the transition function 
# The transition function returns a distribution over next states given the current state and an action .
function T(s::State, a::Action)
    # randomly sample a family size and visa status 
    if s.t == 0
        return Deterministic(params.null_state)
    end
        
    #null_state = params.nullstate
        
    f′ = rand(params.family_sizes, 1)[1]
    v′ = rand(params.visa_status, 1)[1] # TODO: possibly make this weighted in some way 
    #next_states = Vector{State}(undef, Nₐ) 
    if a == ACCEPT 
        Nₐ = 2 #80% get a seat, 20% lost it due to poor organization
        next_states = Vector{State}(undef, Nₐ) 
        probabilities = [.80, .20]
        next_state_accept = State(s.c - s.f, s.t - 1, f′, v′) # they get seats
        next_state_reject = State(s.c, s.t - 1, f′, v′)
        if validcapacity(next_state_accept) 
            next_states[1] = next_state_accept
        else
            probabilities = [0, 1] #no room for full family :( so we make probability 0 to accept and 1 reject
        end
        next_states[2] = next_state_reject  # no need to check capacity if rejecting 

    elseif a == REJECT
        Nₐ = 1
        next_states = Vector{State}(undef, Nₐ) 
        probabilities = [1.0]
        next_states[1] = State(s.c, s.t - 1, f′, v′) #where they don't get seats
    
        # handle out-of-bounds transitions
    else 
        Nₐ = 1
        next_states = Vector{State}(undef, Nₐ) 
        probabilities = [1.0]
        next_states[1] = s
    end
    return SparseCat(next_states, probabilities)
end

    

T (generic function with 1 method)

In [39]:
# struct State
#     c::Int # chairs remaining 
#     t::Int # time remaining 
# #     f::Int # family size 
# #     v::Int # visa status 
# # end 
T(State(6, 4, 4, 5), ACCEPT)


# ACCEPT: State(50, 9, ANYTHING, ANYTHING)
# ACCEPT: State(42, 9, ANYTHING, ANYTHING)
# REJECT: State(50, 9, ANYTHING, ANYTHING)

LoadError: type #params has no field family_sizes

**Reward Function**

In [40]:
function R(s::State, a::Action)
    # reward is just the visa status times family size i think! 
    if a == ACCEPT
        return s.v*s.f
    end
    return 0
end 

R (generic function with 1 method)

In [41]:
#render(mdp; show_rewards=true)
#@bind γ Slider(0:0.05:1, default=0.95, show_value=true)
# hard code for now. maybe come back to it. 
γ = 0.95

0.95

In [42]:
# when time is 0...possibly encode this into the parameters 
#termination(s::State) = (s.t == 0 || s.c == 0)

termination(s::State) = (s.t == params.null_state.t && s.c == params.null_state.c)

termination (generic function with 1 method)

**MDP Formulation**

In [43]:
# We define the Airport abstract MDP type so we can reference it in other methods.
abstract type Evacuation <: MDP{State, Action} end

In [26]:
  # struct State
        #     c::Int # chairs remaining 
        #     t::Int # time remaining 
        #     f::Int # family size 
        #     v::Int # visa status 
        # end 
c_initial = params.capacity
t_initial = params.time
f_initial = rand(params.family_sizes, 1)[1]
v_initial = rand(params.visa_status, 1)[1]

initial_state = State(c_initial, t_initial, f_initial, v_initial)

State(20, 60, 3, 3)

In [27]:
statetype = typeof(initial_state)
@show initialstate_array = [initial_state]

initialstate_array = [initial_state] = State[State(20, 60, 3, 3)]


1-element Vector{State}:
 State(20, 60, 3, 3)

In [28]:
initialstate_array

1-element Vector{State}:
 State(20, 60, 3, 3)

In [15]:
mdp = QuickMDP(Evacuation,
    states       = 𝒮,
    actions      = 𝒜,
    transition   = T,
    reward       = R,
    discount     = γ,
    initialstate = 𝒮, # initialstate_array,
    isterminal   = termination,
    render       = render,
    statetype    = statetype #
    );

LoadError: UndefVarError: 𝒜 not defined

In [30]:
#render(mdp)

In [2]:
solver = ValueIterationSolver(max_iterations=30, belres=1e-6, verbose=true);

**Policy**

In [32]:
# Set discount factor to variable gamma and solve the MDP to obptain policy pi mapping states to actions a

In [3]:
@show policy = solve(solver, mdp) 

LoadError: UndefVarError: mdp not defined

In [ ]:
@requirements_info ValueIterationSolver() mdp()

In [32]:
# a = action(policy, s) # returns the optimal action for state s
# value(policy, s) # returns the optimal value at state s


**Visualizations**

In [ ]:
struct evacPolicy <: Policy end 

In [ ]:
function plot_evac(mdp::MDP,
                   policy::Policy=evacPolicy(), 
                   iter=0)
                   # Robert's gridworld had more parameters, but not sure if we need all of them 

    gr()
    
    (xmax, ymax) = params.size # reshape grid to family size * visa status
    Uxy = reshape(U, xmax, ymax)
    # plot values (i.e the U matrix)
    fig = heatmap(Uxy',
                  legend=:none,
                  aspect_ratio=:equal,
                  framestyle=:box,
                  tickdirection=:out)
    xlims!(0.5, xmax+0.5)
    ylims!(0.5, ymax+0.5)
    xticks!(1:xmax)
    yticks!(1:ymax)
    rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])
    
    # Let x represent family size, y represent visa status 
    for x in 1:xmax, y in 1:ymax
        policy_grid(policy, xmax, ymax)
        annotate!([(x, y, (grid[x,y], :center, 12, "Computer Modern"))])
        rect = rectangle(1, 1, x - 0.5, y - 0.5)
        plot!(rect, fillalpha=0, linecolor=:gray)
        
        # Some if statement here to say: 
        # if policy at State(c,t,f,v) == ACCEPT
            # color = green
        # else 
            # no color 
        return fig 
    end